In [35]:
import pandas as pd
import numpy as np
import torch
import os
from dateutil.parser import parse 
import datetime

In [36]:
country_codes = ['ABW','AFG','AGO','ALB','AND','ARE','ARG','AUS','AUT','AZE','BDI','BEL','BEN','BFA','BGD','BGR','BHR','BHS','BIH','BLR','BLZ','BMU','BOL','BRA','BRB','BRN','BTN','BWA','CAF','CAN','CHE','CHL','CHN','CIV','CMR','COD','COG','COL','COM','CPV','CRI','CUB','CYP','CZE','DEU','DJI','DMA','DNK','DOM','DZA','ECU','EGY','ERI','ESP','EST','ETH','FIN','FJI','FRA','FRO','GAB','GBR','GEO','GHA','GIN','GMB','GRC','GRL','GTM','GUM','GUY','HKG','HND','HRV','HTI','HUN','IDN','IND','IRL','IRN','IRQ','ISL','ISR','ITA','JAM','JOR','JPN','KAZ','KEN','KGZ','KHM','KOR','KWT','LAO','LBN','LBR','LBY','LKA','LSO','LTU','LUX','LVA','MAC','MAR','MCO','MDA','MDG','MEX','MLI','MMR','MNG','MOZ','MRT','MUS','MWI','MYS','NAM','NER','NGA','NIC','NLD','NOR','NPL','NZL','OMN','PAK','PAN','PER','PHL','PNG','POL','PRI','PRT','PRY','PSE','QAT','RKS','ROU','RUS','RWA','SAU','SDN','SEN','SGP','SLB','SLE','SLV','SMR','SOM','SRB','SSD','SUR','SVK','SVN','SWE','SWZ','SYC','SYR','TCD','TGO','THA','TJK','TKM','TLS','TTO','TUN','TUR','TWN','TZA','UGA','UKR','URY','USA','UZB','VEN','VIR','VNM','VUT','YEM','ZAF','ZMB','ZWE']
filenames = ["c1_school_closing.csv", "c2_workplace_closing.csv", "c3_cancel_public_events.csv", "c4_restrictions_on_gatherings.csv", "c5_close_public_transport.csv", "c6_stay_at_home_requirements.csv", "c7_movementrestrictions.csv", "c8_internationaltravel.csv", "confirmed_cases.csv"]

def dateConvertor(date):
    dt = parse(date)
    date = dt.strftime('%Y-%m-%d')
    return date

country_code2id = {}
for i in range(len(country_codes)):
    country_code2id[country_codes[i]] = i 

# date extraction
npi_date = pd.DataFrame({})
npi_date['Date'] = pd.read_csv(os.path.join('timeseries', filenames[0])).keys()[3:]
npi_date['Date'] = npi_date['Date'].apply(dateConvertor)

In [37]:
dataframes = {} 

countries_to_extract = ['ITA','IND','USA','CHN','BRA','IRN','CAN','GBR',
                        'FRA','ESP','BEL','DEU','NLD','MEX','TUR','SWE','ECU','RUS','PER','CHE'] # countries code for which you want data.
countries_to_extract = ['AUS','IND','USA','CHN', 'GBR']
index = [country_code2id[code] for code in countries_to_extract]
static_data = pd.read_csv(os.path.join('timeseries', 'Consolidated.csv')).T[2:][index].T
static_data = static_data.drop('Population', axis = 1 )
cols_to_norm = ['Density', 'Median Age']
static_data[cols_to_norm] = static_data[cols_to_norm].apply(lambda x: (x - x.min()) / x.max()-x.min())
static_data = static_data.to_numpy()
tmp = static_data[:,3:]
final_static_data = static_data[:,0:3].astype(np.float64)
#print(final_static_data)
for file in filenames:
    npi_df = pd.read_csv(os.path.join('timeseries', file)).T[3:]
    npi_df['Date'] = npi_date['Date'].values
    npi_df.set_index('Date', drop=True, inplace=True)
    npi_df = npi_df[index] # selecting countries 
    npi_df = npi_df[64:335] # removing Jan, Feb and Dec data
    for col in npi_df:
        npi_df[col] = pd.to_numeric(npi_df[col], errors='coerce') # converting object to numeric 
    npi_df.interpolate(method='linear', inplace=True) # interpolate missing values 
    dataframes[file[:-4]] = npi_df
   
    
    
    if(file[:-4]=='confirmed_cases'):
#         npi_df = pd.read_csv(os.path.join('timeseries', file))
#         print(npi_df)
        npi_df = pd.read_csv(os.path.join('timeseries', file)).T[3:]
        npi_df['Date'] = npi_date['Date'].values
        npi_df.set_index('Date', drop=True, inplace=True)
        npi_df = npi_df[index] # selecting countries 
#         npi_df = npi_df[64:335] # removing Jan, Feb and Dec data
        for col in npi_df:
            npi_df[col] = pd.to_numeric(npi_df[col], errors='coerce')
        npi_df = npi_df.interpolate(method='linear') # interpolate missing values     
        npi_df = npi_df.rolling(7).mean()
        
        npi_df = 100*npi_df.diff()/npi_df
        npi_df = npi_df[64:335] # removing Jan, Feb and Dec data
        
        
        
#         npi_df.interpolate(method='linear', inplace=True) # interpolate missing values     
        dataframes['growth_rate'] = npi_df
    

[[ -3.202      -17.8399061    0.939     ]
 [ -2.20910894 -18.06760563   0.64      ]
 [ -2.54750274 -17.72957746   0.894     ]
 [ -3.1925044  -18.3          0.367     ]
 [ -3.19166074 -18.18732394   0.702     ]]


In [65]:
def readData(attributes, history, date):
    index = dataframes['c1_school_closing'].index.get_loc(date)
    if(history>index):
        print('Not sufficient history')
        sys.exit()
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    ref_date = date - datetime.timedelta(21)
    if ref_date.month == date.month:
        temperature = tmp[:,int(date.month)].reshape((len(countries_to_extract),1)).astype(np.float64)
    else:
        temperature = ((tmp[:,int(date.month)] + tmp[:,int(ref_date.month)]) / 2).reshape((len(countries_to_extract),1)).astype(np.float64)
    data = []
    past_growthrates = dataframes['growth_rate'].iloc[index-history:index].values.reshape((len(countries_to_extract),21))
    new_final_static_data = np.concatenate((final_static_data,past_growthrates,temperature),axis=1)
    for att in attributes:
        temp = dataframes[att].iloc[index-history:index].values
        if(len(data)==0):
            data = np.asarray(temp)
        else:
            data = np.dstack((data, temp))
    x = torch.cat((torch.from_numpy(data).to(dtype=torch.double).permute(1,0,2).view(len(countries_to_extract),-1),
                   torch.from_numpy(new_final_static_data).to(dtype=torch.double)),dim = -1)
    #x = torch.cat((torch.from_numpy(data).to(dtype=torch.double).permute(1,0,2).view(len(countries_to_extract),-1),
                   #torch.from_numpy(past_growthrates).to(dtype=torch.double)),dim = -1)
    #x = torch.from_numpy(data).to(dtype=torch.double).permute(1,0,2)
    y = torch.from_numpy(dataframes['growth_rate'].iloc[index].values).to(dtype=torch.double)
    return x,y

In [66]:
import torch.nn.functional as F
import torch 
import torch.optim as optim
import sys
import torch.nn as nn
import pandas as pd 
import os
import numpy as np
from dateutil.parser import parse 

In [71]:
class Encoder(torch.nn.Module):
    def __init__(self, input_dim , out_dim = 64):
        super(Encoder, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim,256)
        self.linear2 = torch.nn.Linear(256,512)
        self.linear3 = torch.nn.Linear(512,256)
        self.linear4 = torch.nn.Linear(256,128)
        self.linear5 = torch.nn.Linear(128,out_dim)
        self.prelu1   = torch.nn.PReLU()
        self.prelu2   = torch.nn.PReLU()
        self.prelu3   = torch.nn.PReLU()
        self.prelu4   = torch.nn.PReLU()
        self.tanh    = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(p=0.0)
        
    def forward(self, x):
        x = self.prelu1(self.linear1(x))
        x = self.dropout(self.prelu2(self.linear2(x)))
        x = self.dropout(self.prelu3(self.linear3(x)))
        x = self.dropout(self.prelu4(self.linear4(x)))
        x = self.tanh(self.linear5(x)/20)*24
        return x.squeeze()
    
class Decoder1(torch.nn.Module):
    def __init__(self,input_dim,output_dim=1):
        super(Decoder1, self).__init__()
        
        self.linear1 = torch.nn.Linear(input_dim,32)
        self.linear2 = torch.nn.Linear(32,16)
        self.linear3 = torch.nn.Linear(16,1)
        self.prelu1   = torch.nn.PReLU()
        self.prelu2   = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(p=0.2)
    
    def forward(self,x):
        x = self.dropout(self.prelu1(self.linear1(x)))
        x = self.dropout(self.prelu2(self.linear2(x)))
        x = self.linear3(x)
        return x.squeeze()

class Decoder2(torch.nn.Module):
    def __init__(self,input_dim,output_dim=214):
        super(Decoder2, self).__init__()
        

        self.linear1 = torch.nn.Linear(input_dim,64)
        self.linear2 = torch.nn.Linear(input_dim,128)
        self.linear3 = torch.nn.Linear(128,output_dim)
        self.prelu1   = torch.nn.PReLU()
        self.prelu2   = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(p=0.2)
        self.relu = torch.nn.ReLU()
        
    def forward(self,x):
        x = self.dropout(self.prelu1(self.linear1(x)))
        x = self.dropout(self.prelu2(self.linear2(x)))
        x = self.relu(self.linear3(x))
        return x.squeeze() 

In [72]:
##training,testing and validation split
training_attributes = ["c1_school_closing", "c2_workplace_closing", "c3_cancel_public_events", "c4_restrictions_on_gatherings", "c5_close_public_transport", "c6_stay_at_home_requirements", "c7_movementrestrictions", "c8_internationaltravel", "confirmed_cases"]
history = 21
train_dates = npi_date['Date'][85:300].values
validation_dates = npi_date['Date'][300:335].values
x,y = readData(attributes=training_attributes, history=history, date=train_dates[-1])
print(x.shape)


torch.Size([5, 214])


In [73]:
## Train the predictor first Encoder+ Decoder 1
enc = Encoder(21*9+25)
dec1 = Decoder1(64)
params = list(enc.parameters())+list(dec1.parameters())
optim_1 = optim.Adam(params, lr=1e-4, weight_decay=1e-5)
mse_loss = torch.nn.L1Loss()
best_valid_loss = 10000000000
patience = 0

for epoch in range(100):
    #np.random.shuffle(train_dates)
    epoch_loss = 0
    enc.train()
    dec1.train()
    loss_t = 0.0
    y_gt = []
    y_pred = []
    for i in range(len(train_dates)):
        
        optim_1.zero_grad()
        x,y = readData(attributes=training_attributes, history=history, date=train_dates[i])
        x,y = x.float(), y.float()
        x_temp = x[0].view(-1)
        y_temp = y[0]
        gr_pred = dec1(enc(x_temp))
        loss = mse_loss(gr_pred,y_temp)
        loss.backward()
        optim_1.step()
        loss_t += loss.item()
        y_gt.append('%.2f' % y_temp)
        y_pred.append('%.2f' % gr_pred)
    
    enc.eval()
    dec1.eval() 
    valid_loss = 0
    for i in range(len(validation_dates)):
        x,y = readData(attributes=training_attributes, history=history, date=validation_dates[i])
        x,y = x.float(), y.float()
        x_temp = x[0].view(-1)
        y_temp = y[0]
        features = dec1(enc(x_temp))
        loss = mse_loss(features,y_temp)
        valid_loss += loss.item()
    print(epoch, 'training loss =',loss_t,'validation loss=',valid_loss)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        patience = 0
    else:
        patience +=1
    if patience >=20:
        break

    if (epoch+1)%10 == 0:
        print(y_gt)
        print(y_pred)
        print('='*50)
        
        


0 training loss = 299.43714535981417 validation loss= 1.8700228463858366
1 training loss = 298.2256590947509 validation loss= 0.5964588616043329
2 training loss = 245.32903130725026 validation loss= 4.124207653105259
3 training loss = 303.7439147271216 validation loss= 8.09232459962368
4 training loss = 245.9518735036254 validation loss= 6.466324090957642
5 training loss = 209.75338522717357 validation loss= 6.328000590205193
6 training loss = 197.28697706758976 validation loss= 2.5759029053151608
7 training loss = 252.7118924483657 validation loss= 7.2768916338682175
8 training loss = 200.30786969140172 validation loss= 5.540608778595924
9 training loss = 198.41396910697222 validation loss= 3.81608247756958
['17.29', '16.04', '14.91', '12.38', '11.99', '10.12', '9.13', '7.77', '6.87', '5.66', '4.80', '3.89', '3.49', '2.91', '2.46', '2.14', '1.79', '1.47', '1.28', '1.19', '0.97', '0.80', '0.69', '0.59', '0.65', '0.60', '0.50', '0.46', '0.43', '0.33', '0.27', '0.22', '0.21', '0.21', '0.

In [30]:
## Train the prescribtor  Decoder 2,freeze encoder and decoder 1
## to be written
dec2 = Decoder2(64)
optim_2 = optim.Adam(dec2.parameters(), lr=1e-4, weight_decay=1e-5)
for p in enc.parameters():
     p.requires_grad = False
for p in dec1.parameters():
     p.requires_grad = False
mse_loss = torch.nn.MSELoss(reduction='none')

for epoch in range(1):
    np.random.shuffle(train_dates)
    epoch_loss = 0
    dec2.train()
    for i in range(len(train_dates)):
        optim_2.zero_grad()
        x,y = readData(attributes=training_attributes, history=history, date=train_dates[0])
        x = x.float()
        npi = dec2(enc(x))
        #print(npi.shape)
        features = dec1(enc(npi))
        #print(features.shape)
        break
        

In [6]:
print(train_dates)

NameError: name 'train_dates' is not defined